In [9]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [10]:
def compute_RMS_error(D1, D2):
    e_rms = np.sqrt(np.sum(D1**2 + D2**2) / (2*np.size(D1, 0)))
    return e_rms

In [11]:
def plot_lines_points_and_image(img_pts, img, l, path, save=False):

    fig = plt.figure(figsize=(8,6))
    ax = plt.axes()
    
    cv.compute_and_plot_lines(l, img, ax)
    ax.plot(img_pts[0], img_pts[1], 'o', color='blue', label='Random points')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # plt.gca().invert_yaxis()
    # ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.legend(loc="upper right")
    # ax.margins(x=0.1, y=0.1) 
    # fig.tight_layout()

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if save:
        fig.savefig(path, dpi=300)
    plt.show()


In [12]:
def plot_histogram(data, path, mean, save=False):
    fig = plt.figure()
    plt.hist(data, bins=100, color='tab:blue')
    plt.axvline(mean, linestyle='--', color='red', label='Mean: {}'.format(round(mean, 2)))
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    # plt.xlim([0,70])
    plt.legend(loc="upper right")
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [25]:
# @njit()
def estimate_E_robust(K, x1_norm, x2_norm):
    
    err_threshold_px = 2
    err_threshold = err_threshold_px / K[0,0]
    
    # alpha = 0.95
    # E = cv.estimate_E_DLT(x1_norm, x2_norm, enforce=True, verbose=False)
    # D1, D2, _ = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
    # inliers = ((D1**2 + D2**2) / 2) < err_threshold**2
    # epsilon = np.sum(inliers) / np.size(inliers)
    # s = 8
    # T = np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))

    s = 8

    best_inliers = None
    best_E = None
    best_err = np.inf

    for t in trange(1000):

        rand_mask = np.random.choice(np.size(x1_norm,1), s, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2, _ = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        # F = cv.convert_E_to_F(E, K1, K2)
        # D1, D2, _ = cv.compute_epipolar_errors(F, K1 @ x1_norm, K2 @ x2_norm)
        # inliers = ((D1**2 + D2**2) / 2) < err_threshold**2
        # inliers = (D1 < err_threshold_px) * (D2 < err_threshold_px)

        inlier_err = 1 - (np.sum(inliers) / np.size(inliers))

        if inlier_err < best_err:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            best_err = inlier_err

            print(np.sum(inliers), end='\r')
        
    return best_E, best_inliers


In [48]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex = r'\compEx1data.mat'
img1 = r'\round_church1.jpg'
img2 = r'\round_church2.jpg'
report = r'\report-images'

round_church1 = cv.load_image(path+data+img1)
round_church2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex, 'K')
K_inv = np.linalg.inv(K)
x = cv.convert_mat_to_np(path+data+compex, 'x')
x1 = cv.dehomogenize(x[0,0])
x2 = cv.dehomogenize(x[0,1])

ransac = True
plt_pts = True
plt_hist = True
save = False

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

In [45]:
if ransac:
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm)
    np.save(path+data+'/E_robust_3.npy', E)
    np.save(path+data+'/inliers_3.npy', inliers)
    print('No. inliers:', np.sum(inliers))

    # %snakeviz -t estimate_E_robust(K, x1_norm, x2_norm)
else:
    E = cv.estimate_E_DLT(x1_norm, x2_norm, enforce=True, verbose=False)


In [49]:
if ransac:
    E = np.load(path+data+'/E_robust_2.npy')
inliers = np.load(path+data+'/inliers_2.npy')
F = cv.convert_E_to_F(E, K, K)

print('\nE:', E)
print('F:', F)



# All points
D1, D2, e_rms = cv.compute_epipolar_errors(F, x1, x2)
e_rms = compute_RMS_error(D1, D2)

print('\nAll points:')
print('Ransac:', ransac)
print('Mean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('RMS error:', e_rms)

path1 = path+report+'/CE1_hist1_ransac={}_all_points.png'.format(ransac)
path2 = path+report+'/CE1_hist2_ransac={}_all_points.png'.format(ransac)

if plt_hist:
    plot_histogram(D1, path1, np.mean(D1), save)
    plot_histogram(D2, path2, np.mean(D2), save)



# Inliers
D1, D2, e_rms = cv.compute_epipolar_errors(F, x1[:,inliers], x2[:,inliers])
e_rms = compute_RMS_error(D1, D2)

print('\nInliers:')
print('Ransac:', ransac)
print('Mean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('RMS error:', e_rms)
print('No. inliers:', np.sum(inliers))

path1 = path+report+'/CE1_hist1_ransac={}_inliers.png'.format(ransac)
path2 = path+report+'/CE1_hist2_ransac={}_inliers.png'.format(ransac)

if plt_hist:
    plot_histogram(D1, path1, np.mean(D1), save)
    plot_histogram(D2, path2, np.mean(D2), save)



# Plot
if ransac:
    x1_norm = x1_norm[:,inliers]
    x2_norm = x2_norm[:,inliers]
    x1 = x1[:,inliers]
    x2 = x2[:,inliers]

rand_mask = np.random.choice(np.size(x2_norm,1), 20, replace=False)
x1_rand = x1[:,rand_mask]
x2_rand = x2[:,rand_mask]
l1, l2 = cv.compute_epipolar_lines(F, x1_rand, x2_rand)

path1 = path+report+'/CE1_round_church1_ransac={}.png'.format(ransac)
path2 = path+report+'/CE1_round_church2_ransac={}.png'.format(ransac)

if plt_pts:
    plot_lines_points_and_image(x1_rand, round_church1, l1, path1, save)
    plot_lines_points_and_image(x2_rand, round_church2, l2, path2, save)


E: [[-3.62864329e-02  4.98845083e+01 -8.33500424e+00]
 [-5.43660935e+01 -1.50274051e+00  2.93397860e+01]
 [ 9.50498552e+00 -3.63370854e+01  1.00000000e+00]]
F: [[-3.49509341e-09  4.79650639e-06 -4.93214050e-03]
 [-5.22742077e-06 -1.44240809e-07  1.17181195e-02]
 [ 5.47916592e-03 -1.27457649e-02  1.00000000e+00]]

All points:
Ransac: True
Mean distance 1: 58.74538197965829
Mean distance 2: 71.52507914842866
RMS error: 209.63596816010937

Inliers:
Ransac: True
Mean distance 1: 0.8859416907684401
Mean distance 2: 1.0020265668647925
RMS error: 1.084528021008188
No. inliers: 5231


In [ ]:
# All points:
# Ransac: False
# Mean distance 1: 61.33360616690827
# Mean distance 2: 63.46892741628846
# RMS error: 155.95832290161417

# Inliers:
# Ransac: False
# Mean distance 1: 14.527720653307188
# Mean distance 2: 15.349401111343274
# RMS error: 21.875573131094626
# No. inliers: 5231

# All points:
# Ransac: True
# Mean distance 1: 58.74538197965829
# Mean distance 2: 71.52507914842866
# RMS error: 209.63596816010937

# Inliers:
# Ransac: True
# Mean distance 1: 0.8859416907684401
# Mean distance 2: 1.0020265668647925
# RMS error: 1.084528021008188
# No. inliers: 5231